## Gated Recurrent Unit implementation

In [ ]:
from theano import *
import theano.tensor as T

from sklearn.preprocessing import normalize
from theano.tensor.shared_randomstreams import RandomStreams
import pandas as pd
import numpy as np

## Load data

In [ ]:
tsx = pd.read_csv('SPY.csv')
#differencing and normalizing features
X = tsx.values[:,1:-2]
Xd = np.diff(X,axis=0)
Xsd = ((Xd - np.mean(Xd,axis=0)) / np.var(Xd,axis=0)).astype(np.float32)[:-1][-5365:]

#label
Y = ((Xd[:,4] > 0)*1)[1:].astype(np.int32)[-5365:].reshape(-1,1)
Xsd.shape, Y.shape

## GRU construction

In [ ]:
##parameters
n_in = Xsd.shape[1]
n_blocks = 36
seed = 132345
activation = T.nnet.relu

##computational flow
rng = np.random.RandomState(seed)
x = T.matrix('x')
y = T.imatrix('y')
def genW(size):
    return np.asarray(
        rng.uniform(
            low=-0.1,
            high=0.1,
            size = size
        ),
        dtype = theano.config.floatX)
M = n_in
N = n_blocks

#input block
Wz = theano.shared(value=genW((M,N)),name="Wz",borrow=True)
Wr = theano.shared(value=genW((M,N)),name="Wr",borrow=True)
W = theano.shared(value=genW((M,N)),name="W",borrow=True)
#recurrent block
Uz = theano.shared(value=genW((N,N)),name="Uz",borrow=True)
Ur = theano.shared(value=genW((N,N)),name="Ur",borrow=True)
U = theano.shared(value=genW((N,N)),name="U",borrow=True) 
params = [Wz,Wr,W,Uz,Ur,U]
u0 = theano.shared(value=np.zeros((1,N),dtype=theano.config.floatX),borrow=True)
#regression weights
W_out = theano.shared(value=genW((N,1)),name="W_out",borrow=True)
b_out = theano.shared(value=np.zeros(1,dtype=np.float32),name="b_out",borrow=True)
params += [W_out, b_out]
#recurrence step
def recurrence(x_t,h_t1):
    #update gate
    z_t = T.nnet.sigmoid(T.dot(x_t,Wz) + T.dot(h_t1,Uz))
    #reset gate
    r_t = T.nnet.sigmoid(T.dot(x_t,Wr) + T.dot(h_t1,Ur))
    h_tl = activation(T.dot(x_t,W) + T.dot(r_t*h_t1,U))
    #block output
    h_t = (1-z_t)*h_t1 + z_t*h_tl
    
    return h_t, OrderedDict({u0: h_t})
#hidden state computation
u_t, _ = theano.scan(fn=recurrence,
                     sequences=[x],
                     outputs_info=[u0]
                     )
#hidden state output
gru_out = u_t[:,0,:]
#output probability
p_y_x = T.nnet.sigmoid(T.dot(gru_out,W_out) + b_out)
#label prediction
y_p = T.gt(p_y_x,0.5)*1
#cross entropy loss
CE = T.nnet.binary_crossentropy(p_y_x,y).mean()

## Training

In [ ]:
#parameters
tlength = 5000
learning_rate=0.1
n_epochs=1000
   
sX = theano.shared(np.asarray(Xsd[:tlength], dtype=theano.config.floatX))
sY = theano.shared(np.asarray(Y[:tlength], dtype=np.int32))


####
#training function
cost = CE
gparams = [T.grad(cost,param) for param in params]
updates = [(param, param - learning_rate * gparam) for param,gparam in zip(params,gparams)]
train_gru = theano.function(
    inputs=[],
    outputs=[cost],
    updates=updates,
    givens={
        x: sX,
        y: sY
    },
)
#prediction function
pred_y = theano.function(
    inputs=[x],
    outputs=y_p
)

In [ ]:
#training
print "training..."
for ep in range(n_epochs):
        print('Training epoch %d, cost %f' % (ep, np.mean(train_gru())))

## Testing

In [ ]:
print np.mean(pred_y(Xsd)[tlength:]==Y[tlength:])